# Semantic Layer Demo - DuckDB Version (No Java Required!)

This notebook demonstrates the semantic layer using **DuckDB** instead of PySpark. 
DuckDB requires no Java and runs entirely in the Colab environment.

**Advantages of this approach:**
- ✅ No Java installation needed
- ✅ Runs completely in Colab
- ✅ Fast in-memory SQL queries
- ✅ Perfect for data exploration

In [ ]:
# Install dependencies
!pip install -q duckdb pandas

print("✓ Dependencies installed")

## Step 1: Create Sample Data

We'll create sample customers and transactions data, then build the gold semantic layer using DuckDB.

In [ ]:
import duckdb
import pandas as pd
from datetime import datetime, timedelta
import random

# Create in-memory DuckDB connection
conn = duckdb.connect(':memory:')

# Sample customers
customers_data = {
    'customer_id': ['c_001', 'c_002', 'c_003', 'c_004', 'c_005'],
    'email': ['alice@example.com', 'bob@example.com', 'charlie@example.com', 
              'diana@example.com', 'eve@example.com']
}
customers_df = pd.DataFrame(customers_data)

# Sample transactions
transactions_data = {
    'transaction_id': ['t_001', 't_002', 't_003', 't_004', 't_005', 't_006', 't_007', 't_008', 't_009', 't_010'],
    'customer_id': ['c_001', 'c_001', 'c_001', 'c_002', 'c_002', 'c_003', 'c_004', 'c_004', 'c_004', 'c_005'],
    'amount': [45.50, 50.00, 49.99, 150.00, 150.00, 75.25, 100.00, 120.00, 110.00, 80.00]
}
transactions_df = pd.DataFrame(transactions_data)

# Register as DuckDB tables
conn.execute("CREATE TABLE customers AS SELECT * FROM customers_df")
conn.execute("CREATE TABLE transactions AS SELECT * FROM transactions_df")

print("✓ Sample data created\n")
print("Customers:")
print(customers_df)
print("\nTransactions:")
print(transactions_df)

## Step 2: Build Gold Semantic Layer

Create aggregated metrics per customer using DuckDB SQL.

In [7]:
# Build gold view (semantic layer) using DuckDB

gold_view_sql = """
SELECT 
    t.customer_id,
    SUM(t.amount) as total_spend,
    COUNT(t.transaction_id) as transaction_count,
    ROUND(AVG(t.amount), 2) as avg_transaction_amount
FROM transactions t
GROUP BY t.customer_id
ORDER BY total_spend DESC
"""

gold_df = conn.execute(gold_view_sql).fetch_df()

# Register gold view in DuckDB
conn.execute("CREATE TABLE gold_view AS " + gold_view_sql)

print("✓ Gold semantic layer created\n")
print("Gold View (Semantic Layer):")
print(gold_df)

NameError: name 'conn' is not defined

## Step 3: Run Example Queries

Explore the gold layer with various SQL queries.

In [ ]:
# Query 1: Top 3 customers by spending
print("=" * 60)
print("TOP 3 CUSTOMERS BY SPENDING")
print("=" * 60)
result = conn.execute("""
    SELECT 
        customer_id,
        total_spend,
        transaction_count,
        avg_transaction_amount
    FROM gold_view
    ORDER BY total_spend DESC
    LIMIT 3
""").fetch_df()
print(result.to_string(index=False))

# Query 2: Summary statistics
print("\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)
result = conn.execute("""
    SELECT 
        COUNT(*) as total_customers,
        SUM(transaction_count) as total_transactions,
        ROUND(SUM(total_spend), 2) as total_revenue,
        ROUND(AVG(total_spend), 2) as avg_customer_spend,
        ROUND(AVG(avg_transaction_amount), 2) as avg_transaction_value
    FROM gold_view
""").fetch_df()
print(result.to_string(index=False))

# Query 3: Spending distribution
print("\n" + "=" * 60)
print("CUSTOMER SEGMENTS BY SPENDING")
print("=" * 60)
result = conn.execute("""
    SELECT 
        CASE 
            WHEN total_spend < 100 THEN 'Low Spender'
            WHEN total_spend < 200 THEN 'Mid Spender'
            ELSE 'High Spender'
        END as segment,
        COUNT(*) as customer_count,
        ROUND(AVG(total_spend), 2) as avg_spend
    FROM gold_view
    GROUP BY segment
    ORDER BY avg_spend DESC
""").fetch_df()
print(result.to_string(index=False))

## Step 4: Export Gold View as CSV

Save the semantic layer for use in downstream applications.

In [ ]:
# Export gold view
gold_df.to_csv('gold_view.csv', index=False)
print("✓ Gold view exported to gold_view.csv\n")
print("First 10 rows:")
print(gold_df.head(10).to_string(index=False))

# Also show as downloadable file in Colab
from google.colab import files
files.download('gold_view.csv')

## Step 5: Advanced Queries

Run more complex analytical queries on the gold layer.

In [ ]:
# Query 4: Find outliers (high-value customers)
print("=" * 60)
print("HIGH-VALUE CUSTOMERS (>90th PERCENTILE)")
print("=" * 60)
result = conn.execute("""
    WITH percentiles AS (
        SELECT APPROX_QUANTILE(total_spend, 0.90) as p90
        FROM gold_view
    )
    SELECT 
        customer_id,
        total_spend,
        transaction_count,
        avg_transaction_amount
    FROM gold_view
    WHERE total_spend >= (SELECT p90 FROM percentiles)
    ORDER BY total_spend DESC
""").fetch_df()
print(result.to_string(index=False))

# Query 5: Transaction frequency analysis
print("\n" + "=" * 60)
print("CUSTOMERS BY TRANSACTION FREQUENCY")
print("=" * 60)
result = conn.execute("""
    SELECT 
        CASE 
            WHEN transaction_count = 1 THEN 'One-time'
            WHEN transaction_count BETWEEN 2 AND 3 THEN '2-3 times'
            ELSE '4+ times'
        END as frequency,
        COUNT(*) as customer_count,
        ROUND(AVG(total_spend), 2) as avg_spend
    FROM gold_view
    GROUP BY frequency
    ORDER BY customer_count DESC
""").fetch_df()
print(result.to_string(index=False))

## Step 6: Visualization

Create visualizations of the semantic layer data.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Semantic Layer Analysis Dashboard', fontsize=16, fontweight='bold')

# 1. Top customers by spending
ax1 = axes[0, 0]
top_5 = conn.execute("SELECT customer_id, total_spend FROM gold_view ORDER BY total_spend DESC LIMIT 5").fetch_df()
ax1.barh(top_5['customer_id'], top_5['total_spend'], color='steelblue')
ax1.set_xlabel('Total Spend ($)')
ax1.set_title('Top 5 Customers by Spend')
ax1.grid(axis='x', alpha=0.3)

# 2. Transaction count distribution
ax2 = axes[0, 1]
ax2.hist(gold_df['transaction_count'], bins=5, color='coral', edgecolor='black')
ax2.set_xlabel('Transaction Count')
ax2.set_ylabel('Number of Customers')
ax2.set_title('Transaction Frequency Distribution')
ax2.grid(axis='y', alpha=0.3)

# 3. Spending vs Transaction Count
ax3 = axes[1, 0]
ax3.scatter(gold_df['transaction_count'], gold_df['total_spend'], s=100, color='green', alpha=0.6)
ax3.set_xlabel('Transaction Count')
ax3.set_ylabel('Total Spend ($)')
ax3.set_title('Spending vs Transaction Frequency')
ax3.grid(True, alpha=0.3)

# 4. Average transaction amount
ax4 = axes[1, 1]
ax4.bar(gold_df['customer_id'], gold_df['avg_transaction_amount'], color='purple', alpha=0.7)
ax4.set_xlabel('Customer ID')
ax4.set_ylabel('Avg Transaction Amount ($)')
ax4.set_title('Average Transaction Value per Customer')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Dashboard visualization created")

## Summary

You've successfully:
1. ✅ Created sample customer and transaction data
2. ✅ Built a semantic gold layer with DuckDB SQL
3. ✅ Ran multiple analytical queries
4. ✅ Exported the gold view as CSV
5. ✅ Created visualizations

### Next Steps:
- Use the exported `gold_view.csv` in your applications
- Modify queries to match your data schema
- Create additional semantic layers for other domains
- Schedule regular updates using DuckDB in your pipeline

### Resources:
- [DuckDB Documentation](https://duckdb.org/docs/)
- [SQL Tutorial](https://www.w3schools.com/sql/)
- [Project Repository](https://github.com/AdarshInturi0425/AI-Project)